In [7]:
import pandas as pd
from LCS import LCS, predicting
# from backend.models import Task
import os

EDU_SYS = {
    "學士" : "學士班",
    "碩士" : "碩士班",
    "碩士專班" : "碩士在職專班",
    "博士" : "博士班"
}

FILTER = ["學制班別", "年級"]

EDU_SYS_DEPTS = {
    
}

STUDENT_HEAD = [
    "college_name",
    "dept_name",
    "education_system",
    "stu_no",
    "stu_name",
    "stu_year",
    "gender",
    "country_name"
]

class Row:
    
    def __init__(self, configure):
        # print(configure)
        # self.col = col
        self.configure = configure
        self.area = {}
    
    def append(self, student):
        pass

    


class Table4_1(Row):
    def __init__(self, configure, country_code):
        super().__init__(configure)
        self.country_code = country_code

    def append(self, values:pd.Series):
        country_code = self.country_code[self.country_code["國別"] == values["country_name"]].國別代碼.values[0]
        if not country_code in self.area:
            self.area[country_code] = {}
            self.area[country_code]["0"] = {}
            self.area[country_code]["1"] = {}
            
            self.area[country_code]["0"]["boy"] = []
            self.area[country_code]["0"]["girl"] = []

            self.area[country_code]["1"]["girl"] = []
            self.area[country_code]["1"]["boy"] = []


        if int(values["gender_code"]) == 1:
            if(country_code == 60 or country_code == 70):
                self.area[country_code]["1"]["boy"].append(values)
            else:
                self.area[country_code]["0"]["boy"].append(values)
        else:
            if(country_code == 60 or country_code == 70):
                self.area[country_code]["1"]["girl"].append(values)
            else:
                self.area[country_code]["0"]["girl"].append(values)


        # print(self.area)

    def output(self):
        outputData = []
        for key, item in self.area.items():
            for j in ("0", "1"):
                configure = self.configure.copy()
                configure["僑居地國別"] = key
                configure["僑生及港澳生畢業學生人數_男"] = len(item[j]["boy"])
                configure["僑生及港澳生畢業學生人數_女"] = len(item[j]["girl"])
                students = []
                for i in item[j]["boy"]:
                    students.append(list(i.get(STUDENT_HEAD).values))
                
                for i in item[j]["girl"]:
                    students.append(list(i.get(STUDENT_HEAD).values))
            
                outputData.append([list(configure.values), students])
            
        # print(test)
        return outputData

    def outputMainTable(self):
        outputData = []
        for key, item in self.area.items():
            for j in ("0", "1"):
                configure = self.configure.copy()
                configure["僑居地國別/地區/省市代碼"] = key
                configure["身分類別代碼"] = j
                configure["僑生及港澳生畢業學生人數_男"] = len(item[j]["boy"])
                configure["僑生及港澳生畢業學生人數_女"] = len(item[j]["girl"])
                
                if(len(item[j]["boy"]) != 0 or len(item[j]["girl"]) != 0):
                    outputData.append(configure)
        
        return outputData


# {
#     "學士班" : {
#         "數學系" : {
#             1 : Row()
#         }
#     }
# }

def constructDataStructure(config:pd.DataFrame, country_code):
    data = {}
    for i in range(len(config.values)):
        dept = config.loc[i]["單位"]
        edu_system = config.loc[i]["學制班別"]

        if not edu_system in data:
            data[edu_system] = {}
        
        if not dept in data[edu_system]:
            data[edu_system][dept] = Table4_1(config.loc[i], country_code)
        
#         if not grade in data[edu_system][dept]:
#             data[edu_system][dept][grade] = Table4(config.loc[i])

    # print(data.keys())

    return data



def cleanData(studentData:pd.DataFrame):
    studentData.education_system = studentData.education_system.str.strip(' ')
    studentData.country_name = studentData.country_name.str.strip(' ')
    studentData.dept_name = studentData.dept_name.str.replace(r'\w士班', '')
    studentData.dept_name = studentData.dept_name.str.replace(r'應用數學','')
    studentData.dept_name = studentData.dept_name.str.replace(r' ', '')
    studentData.stu_name = studentData.stu_name.str.strip(r' ')
    studentData.college_name = studentData.college_name.str.strip(r' ')
    studentData.fillna('', inplace=True)
    return studentData


def init(config):
    global EDU_SYS_DEPTS
    EDU_SYS_DEPTS["學士"] = list(set(config.query("學制班別 == '學士班'").單位))
    EDU_SYS_DEPTS["碩士"] = list(set(config.query("學制班別 == '碩士班'").單位))
    EDU_SYS_DEPTS["博士"] = list(set(config.query("學制班別 == '博士班'").單位))
    EDU_SYS_DEPTS["碩士專班"] = list(set(config.query("學制班別 == '碩士在職專班'").單位))



"""

"""
def main(config, studentData):
#     config = pd.read_excel(file1, dtype=str)
    config.fillna('', inplace=True)
    country_code = pd.read_excel("./國別代碼對照表 (6).xls")
    dataStructure = constructDataStructure(config, country_code)
#     studentData = pd.read_excel(file2, dtype=str)
    init(config)
    studentData = cleanData(studentData)

    for i in range(len(studentData.values)):
        edu_sys = EDU_SYS[studentData.loc[i]["education_system"]]
        dept_name = predicting(studentData.loc[i]["dept_name"], EDU_SYS_DEPTS[studentData.loc[i]["education_system"]])
#         stu_year = studentData.loc[i]["stu_year"]
        try:
            dataStructure[edu_sys][ dept_name ].append(studentData.loc[i])
        except:
            print(dataStructure[edu_sys][dept_name].area)
            raise KeyError

    statisticData = []
    tableData = []
    for i in dataStructure:
        for j in dataStructure[i]:
            statisticData += dataStructure[i][j].output()
            tableData += dataStructure[i][j].outputMainTable()


    newdf = pd.DataFrame(tableData)

    head = list(config.columns)
    head.append('僑居地國別')

    

    return {
        "head" : head,
        "data" : statisticData,
        "sub_head" : STUDENT_HEAD
    }, newdf

In [8]:
config = pd.read_excel("./第10910期_學4-1.僑生及港澳生畢業學生資料統計表.xls")

In [9]:
studentData = pd.read_excel("./108畢業生﹍1090924下午for 學4-1 5-1.xlsx", sheet_name=2)

In [10]:
result = main(config, studentData)

In [11]:
result[1].to_excel("output4-1t2.xlsx")

In [6]:
!open .